In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import pandas as pd
import tensorflow as tf
import sqlite3

conn = sqlite3.connect('../HomelessData.db')
c = conn.cursor()

# Import our input dataset
df_2018 = pd.read_sql("SELECT * FROM CocData WHERE year = 2018", con = conn)

df_2018.head()

,Coc,Total,Emergency,Transitional,SafeHaven,RapidRehousing,PermanentSupportive,PermanentOther,Year,CocYear,Total:1,Unsheltered,Sheltered
0,AK-500,1028,749,279,0,125,604,71,2018,AK-500 2018,1094,94,1000
1,AK-501,912,649,263,0,90,344,0,2018,AK-501 2018,922,215,707
2,AL-500,911,569,308,34,191,1768,0,2018,AL-500 2018,901,219,682
3,AL-501,435,298,137,0,60,203,0,2018,AL-501 2018,551,233,318
4,AL-502,210,102,108,0,0,50,0,2018,AL-502 2018,256,22,234


In [2]:
df_2018=df_2018.drop(['Total:1', 'Sheltered', 'Total', 'CocYear', 'Coc', 'Year'], axis=1)

In [3]:
df_2018.head()

,Emergency,Transitional,SafeHaven,RapidRehousing,PermanentSupportive,PermanentOther,Unsheltered
0,749,279,0,125,604,71,94
1,649,263,0,90,344,0,215
2,569,308,34,191,1768,0,219
3,298,137,0,60,203,0,233
4,102,108,0,0,50,0,22


In [8]:
df_2018.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 391 entries, 0 to 390
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype
---  ------               --------------  -----
 0   Emergency            391 non-null    int64
 1   Transitional         391 non-null    int64
 2   SafeHaven            391 non-null    int64
 3   RapidRehousing       391 non-null    int64
 4   PermanentSupportive  391 non-null    int64
 5   PermanentOther       391 non-null    int64
 6   Unsheltered          391 non-null    int64
dtypes: int64(7)
memory usage: 21.5 KB


In [9]:
df_2018.describe()

,Emergency,Transitional,SafeHaven,RapidRehousing,PermanentSupportive,PermanentOther,Unsheltered
count,391.000000,391.000000,391.000000,391.000000,391.000000,391.000000,391.000000
mean,726.565217,256.278772,6.048593,277.053708,917.396419,94.005115,496.529412
std,3837.501189,423.917634,20.230979,458.457307,2131.329836,336.334525,2046.107412
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,136.000000,54.500000,0.000000,35.000000,143.000000,0.000000,38.500000
50%,255.000000,120.000000,0.000000,121.000000,367.000000,0.000000,107.000000
75%,564.000000,271.500000,0.000000,341.500000,889.500000,46.500000,384.000000
max,74543.000000,4393.000000,255.000000,5428.000000,29266.000000,3092.000000,37570.000000


In [13]:
# Remove homeless outcome target from features data
y = df_2018.Unsheltered.values
X = df_2018.drop(columns="Unsheltered").values

# Split training/test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [14]:
# Preprocess numerical data for neural network

# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [17]:
# Define the deep learning model 
nn_model = tf.keras.models.Sequential()
nn_model.add(tf.keras.layers.Dense(units=16, activation="sigmoid", input_dim=6))
nn_model.add(tf.keras.layers.Dense(units=16, activation="relu"))
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=50)

# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/50
10/10 [==============================] - 0s 2ms/step - loss: -317.1951 - accuracy: 0.0068
Epoch 2/50
10/10 [==============================] - 0s 2ms/step - loss: -429.3182 - accuracy: 0.0068
Epoch 3/50
10/10 [==============================] - 0s 2ms/step - loss: -538.4257 - accuracy: 0.0068
Epoch 4/50
10/10 [==============================] - 0s 2ms/step - loss: -634.5901 - accuracy: 0.0068
Epoch 5/50
10/10 [==============================] - 0s 2ms/step - loss: -751.1976 - accuracy: 0.0068
Epoch 6/50
10/10 [==============================] - 0s 2ms/step - loss: -872.0621 - accuracy: 0.0068 
Epoch 7/50
10/10 [==============================] - 0s 2ms/step - loss: -1019.0052 - accuracy: 0.0068
Epoch 8/50
10/10 [==============================] - 0s 2ms/step - loss: -1168.8391 - accuracy: 0.0068
Epoch 9/50
10/10 [==============================] - 0s 2ms/step - loss: -1378.1609 - accuracy: 0.0068
Epoch 10/50
10/10 [==============================] - 0s 2ms/step - loss: -1613.9857 - a